In [20]:
# script created by LLM using symbolic composition rules to generate compatible data
import random
import csv

# List of keywords in the C language
keywords = ['auto', 'double', 'int', 'struct', 'break', 'else', 'long', 'switch', 'case', 'enum', 'register', 'typedef',
            'char', 'extern', 'return', 'union', 'const', 'float', 'short', 'unsigned', 'continue', 'for', 'signed',
            'void', 'default', 'goto', 'sizeof', 'volatile', 'do', 'if', 'static', 'while']

# List of data types for variables
data_types = ['char', 'int', 'float', 'double', 'void']

# List of labels for comments
comment_labels = ['Useful', 'Not Useful']

# Function to generate a random valid identifier
def generate_identifier():
    first_char = random.choice('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ_')
    rest_chars = ''.join(random.choices('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_', k=random.randint(0, 10)))
    return first_char + rest_chars

# Function to generate a random valid line of code
def generate_line_of_code():
    keyword_or_data_type = random.choice(keywords + data_types)
    identifier = generate_identifier()
    value = random.choice(['', f' = {random.randint(0, 100)}'])
    return f'{keyword_or_data_type} {identifier}{value};'

# Function to generate a random comment
def generate_comment():
    level_of_detail = random.choice(['', ' // ' + ' '.join(generate_identifier() for _ in range(random.randint(1, 5)))])
    return random.choice(['', '/* ' + generate_identifier() + ' */']) + level_of_detail

# Function to generate a useful comment for a given line of code
def generate_useful_comment(line_of_code):
    purpose_keywords = ['Declaration', 'Initialization', 'Calculation', 'Function', 'Definition', 'Usage', 'Explanation']
    variable_keywords = ['Variable', 'Value', 'Data', 'Result', 'Parameter']

    purpose = random.choice(purpose_keywords)
    variable = random.choice(variable_keywords)

    return f'// {purpose} of {variable} in the line of code:\n// {line_of_code}'

# Function to generate a random label for a comment
def generate_comment_label():
    return random.choice(comment_labels)

# Generate 5000 lines of code, comments, and labels
data = []
for _ in range(5000):
    line_of_code = generate_line_of_code()
    comment = generate_comment()
    label = generate_comment_label()

    # Ensure the comment is useful if labeled as Useful
    if label == 'Useful':
        comment = generate_useful_comment(line_of_code)

    data.append((line_of_code, comment, label))

# Function to write data to a CSV file
def write_to_csv(file_path, data):
    with open(file_path, mode='w', newline='') as csv_file:
        fieldnames = ['Line of Code', 'Comment', 'Class']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow({'Line of Code': row[0], 'Comment': row[1], 'Class': row[2]})

# Specify the file path
csv_file_path = 'test.csv'

# Write data to the CSV file
write_to_csv(csv_file_path, data)

print(f'Data has been generated and saved to {csv_file_path}')


Data has been generated and saved to test.csv


In [21]:
import pandas as pd

data = pd.read_csv('FIRE2023_IRSE_training_Code_Comment_Seed_Data.csv')
data.head(5)

,Comments,Surrounding Code Context,Class
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful


In [22]:
# experiment 2: Try self-training with ChatGPT generated data
chatgpt_data = pd.read_csv('chatgpt_generated_data.csv')
chatgpt_data.head(5)

,Line of Code,Comment,Class
0,int randomNumber = rand() % 100 + 1;,"""Generate a random number between 1 and 100.""",Useful
1,int length = strlen(inputString);,"""Get the length of the input string.""",Useful
2,for (int i = 0; i < 10; i++) {,"""Loop from 0 to 9.""",Useful
3,"printf(%d "", i);""","""Print the current value of i.""",Useful
4,"char name[20] = ""John Smith"";","""Declare and initialize a character array to ...",Useful


In [23]:
# experiment 3: Try self-training with ChatGPT symbolically generated data
symbolic_data = pd.read_csv('llm_symbolic_generated_data.csv')
symbolic_data.head(5)

,Line of Code,Comment,Class
0,do rf;,NaN,Not Useful
1,double eVqWsc4JTj;,/* jPMSXf9g7zS */ // A VQL2 YTCzrq,Not Useful
2,break jeckm = 82;,// gIyPYWK0G6 sH7k8dX8n nb8F,Not Useful
3,static q3GV0wdM = 73;,NaN,Not Useful
4,float hmGmMaom5j;,// Function of Parameter in the line of code:\...,Useful


In [24]:
# clean dataset by removing extra quotation marks that ChatGPT adds when it generates content
chatgpt_data["Line of Code"] = chatgpt_data["Line of Code"].str.lstrip('"')
chatgpt_data["Line of Code"] = chatgpt_data["Line of Code"].str.rstrip('"')
# rename columns to match original data columns
chatgpt_data = chatgpt_data.rename(columns={'Comment': 'Comments', 'Line of Code': 'Surrounding Code Context', 'Class': 'Class'})
# change column order to match original dataset
chatgpt_data = chatgpt_data[['Comments', 'Surrounding Code Context', 'Class']]
chatgpt_data['Comments'] = chatgpt_data['Comments'].map(lambda x: x.strip().lstrip('"').rstrip('"'))
chatgpt_data['Surrounding Code Context'] = chatgpt_data['Surrounding Code Context'].map(lambda x: x.strip())
chatgpt_data['Class'] = chatgpt_data['Class'].map(lambda x: x.strip())
chatgpt_data.head(5)

# do the same for the symbolic generated data
symbolic_data["Line of Code"] = symbolic_data["Line of Code"].str.lstrip('"')
symbolic_data["Line of Code"] = symbolic_data["Line of Code"].str.rstrip('"')
# rename columns to match original data columns
symbolic_data = symbolic_data.rename(columns={'Comment': 'Comments', 'Line of Code': 'Surrounding Code Context', 'Class': 'Class'})
# change column order to match original dataset
symbolic_data = symbolic_data[['Comments', 'Surrounding Code Context', 'Class']]
symbolic_data['Comments'] = symbolic_data['Comments'].map(lambda x: str(x).strip().lstrip('"').rstrip('"'))
symbolic_data['Surrounding Code Context'] = symbolic_data['Surrounding Code Context'].map(lambda x: x.strip())
symbolic_data['Class'] = symbolic_data['Class'].map(lambda x: x.strip())
symbolic_data.head(5)

,Comments,Surrounding Code Context,Class
0,nan,do rf;,Not Useful
1,/* jPMSXf9g7zS */ // A VQL2 YTCzrq,double eVqWsc4JTj;,Not Useful
2,// gIyPYWK0G6 sH7k8dX8n nb8F,break jeckm = 82;,Not Useful
3,nan,static q3GV0wdM = 73;,Not Useful
4,// Function of Parameter in the line of code:\...,float hmGmMaom5j;,Useful


In [25]:
# now merge new dataset with the old
full_data = data.append(chatgpt_data)
full_data = full_data.append(symbolic_data)
print(full_data.shape)
full_data.head(5)

(16872, 3)


<ipython-input-25-c6d6eabdf6e6>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = data.append(chatgpt_data)
<ipython-input-25-c6d6eabdf6e6>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = full_data.append(symbolic_data)


,Comments,Surrounding Code Context,Class
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful


In [26]:
# now re-apply data transformation on entire dataset and prepare for training
full_data.loc[:,["Class"]] = pd.get_dummies(full_data.Class,drop_first=True).values

full_data["Input"] = full_data["Comments"] + " " + full_data["Surrounding Code Context"]

<ipython-input-26-f5789c578e43>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  full_data.loc[:,["Class"]] = pd.get_dummies(full_data.Class,drop_first=True).values


In [7]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.4 MB/s eta 0:00:00


In [27]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("flax-sentence-embeddings/st-codesearch-distilroberta-base")

# Encode our code into the vector space
code_emb = model.encode(full_data["Input"].astype(str).values.tolist(), convert_to_tensor=True)

In [9]:
# use SMOTE to balance dataset
# and generate enough "Not Useful" comment data
!pip install imbalanced-learn

In [28]:
from collections import Counter
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
Y = full_data['Class']
Y=Y.astype('int')
X = code_emb.tolist()
counter = Counter(Y)
print(counter)
X, y = oversample.fit_resample(X, Y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({1: 10047, 0: 6825})
Counter({0: 10047, 1: 10047})


In [33]:
# classification report on CV with VotingClassifier
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, make_scorer

# prepare candidate classifiers
rf_model = RandomForestClassifier(random_state = 42)
nn_model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)
svc_model = LinearSVC(random_state = 42)

# define model
model =  VotingClassifier(estimators=[('rf', rf_model), ('nn', nn_model), ('svc', svc_model)], voting='hard')

classification_report_list = []
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in cv.split(X, y):
    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("*** K-Fold Split Results ***")
    classification_report_results = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report_results)
    classification_report_list.append(classification_report_results)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.9331941544885177, 'recall': 0.8895522388059701, 'f1-score': 0.9108507386653082, 'support': 1005}, '1': {'precision': 0.8944866920152091, 'recall': 0.936318407960199, 'f1-score': 0.9149246475449684, 'support': 1005}, 'accuracy': 0.9129353233830846, 'macro avg': {'precision': 0.9138404232518634, 'recall': 0.9129353233830846, 'f1-score': 0.9128876931051383, 'support': 2010}, 'weighted avg': {'precision': 0.9138404232518634, 'recall': 0.9129353233830846, 'f1-score': 0.9128876931051384, 'support': 2010}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.911, 'recall': 0.9064676616915422, 'f1-score': 0.9087281795511223, 'support': 1005}, '1': {'precision': 0.906930693069307, 'recall': 0.9114427860696518, 'f1-score': 0.909181141439206, 'support': 1005}, 'accuracy': 0.908955223880597, 'macro avg': {'precision': 0.9089653465346534, 'recall': 0.908955223880597, 'f1-score': 0.9089546604951642, 'support': 2010}, 'weighted avg': {'precision': 0.9089653465346536, 'recall': 0.908955223880597, 'f1-score': 0.9089546604951642, 'support': 2010}}
*** K-Fold Split Results ***
{'0': {'precision': 0.9107142857142857, 'recall': 0.9134328358208955, 'f1-score': 0.9120715350223547, 'support': 1005}, '1': {'precision': 0.9131736526946108, 'recall': 0.9104477611940298, 'f1-score': 0.9118086696562033, 'support': 1005}, 'accuracy': 0.9119402985074627, 'macro avg': {'precision': 0.9119439692044482, 'recall': 0.9119402985074627, 'f1-score': 0.911940102339279, 'support': 2010}, 'weighted avg': {'precision': 0.911

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

*** K-Fold Split Results ***
{'0': {'precision': 0.9201596806387226, 'recall': 0.9183266932270916, 'f1-score': 0.9192422731804586, 'support': 1004}, '1': {'precision': 0.9185700099304865, 'recall': 0.9203980099502488, 'f1-score': 0.9194831013916502, 'support': 1005}, 'accuracy': 0.9193628670980587, 'macro avg': {'precision': 0.9193648452846046, 'recall': 0.9193623515886702, 'f1-score': 0.9193626872860544, 'support': 2009}, 'weighted avg': {'precision': 0.9193644496472954, 'recall': 0.9193628670980587, 'f1-score': 0.9193627472233892, 'support': 2009}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.9152542372881356, 'recall': 0.9143426294820717, 'f1-score': 0.914798206278027, 'support': 1004}, '1': {'precision': 0.9145129224652088, 'recall': 0.9154228855721394, 'f1-score': 0.9149676777722527, 'support': 1005}, 'accuracy': 0.9148830263812843, 'macro avg': {'precision': 0.9148835798766721, 'recall': 0.9148827575271055, 'f1-score': 0.9148829420251399, 'support': 2009}, 'weighted avg': {'precision': 0.9148833953782096, 'recall': 0.9148830263812843, 'f1-score': 0.9148829842032121, 'support': 2009}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.9261133603238867, 'recall': 0.9113545816733067, 'f1-score': 0.9186746987951808, 'support': 1004}, '1': {'precision': 0.9128305582761999, 'recall': 0.9273631840796019, 'f1-score': 0.9200394866732478, 'support': 1005}, 'accuracy': 0.9193628670980587, 'macro avg': {'precision': 0.9194719593000433, 'recall': 0.9193588828764543, 'f1-score': 0.9193570927342143, 'support': 2009}, 'weighted avg': {'precision': 0.9194686534757406, 'recall': 0.9193628670980587, 'f1-score': 0.9193574324026758, 'support': 2009}}


In [34]:
# calculate average metrics for submission
import numpy
final_dict = {}
final_dict['0-precision'] = []
final_dict['0-recall'] = []
final_dict['0-f1'] = []
final_dict['1-precision'] = []
final_dict['1-recall'] = []
final_dict['1-f1'] = []
final_dict['accuracy'] = []

for dictionary in classification_report_list:
    for key, inner_dict in dictionary.items():
        if key == '0':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['0-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['0-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['0-f1'].append(inner_dict['f1-score'])
        if key == '1':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['1-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['1-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['1-f1'].append(inner_dict['f1-score'])
        if key == "accuracy":
            final_dict['accuracy'].append(dictionary['accuracy'])


for key in final_dict.keys():
    final_dict[key] = numpy.average(final_dict[key])

print("******** VC FINAL DICT ********")
print(final_dict)

******** VC FINAL DICT ********
{'0-precision': 0.9190038995807038, '0-recall': 0.9085305543993182, '0-f1': 0.9136714284181721, '1-precision': 0.9097070513899418, '1-recall': 0.9198442383038328, '1-f1': 0.9146848038468979, 'accuracy': 0.9141871198181978}


In [29]:
# classification report on CV with VotingClassifier
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, make_scorer

# define model
model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)

classification_report_list = []
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in cv.split(X, y):
    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("*** K-Fold Split Results ***")
    classification_report_results = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report_results)
    classification_report_list.append(classification_report_results)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.9164149043303121, 'recall': 0.9054726368159204, 'f1-score': 0.9109109109109109, 'support': 1005}, '1': {'precision': 0.9065880039331367, 'recall': 0.9174129353233831, 'f1-score': 0.9119683481701286, 'support': 1005}, 'accuracy': 0.9114427860696518, 'macro avg': {'precision': 0.9115014541317243, 'recall': 0.9114427860696517, 'f1-score': 0.9114396295405198, 'support': 2010}, 'weighted avg': {'precision': 0.9115014541317245, 'recall': 0.9114427860696518, 'f1-score': 0.9114396295405197, 'support': 2010}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8903846153846153, 'recall': 0.9213930348258706, 'f1-score': 0.9056234718826406, 'support': 1005}, '1': {'precision': 0.9185567010309278, 'recall': 0.8865671641791045, 'f1-score': 0.9022784810126582, 'support': 1005}, 'accuracy': 0.9039800995024876, 'macro avg': {'precision': 0.9044706582077715, 'recall': 0.9039800995024876, 'f1-score': 0.9039509764476494, 'support': 2010}, 'weighted avg': {'precision': 0.9044706582077716, 'recall': 0.9039800995024876, 'f1-score': 0.9039509764476493, 'support': 2010}}
*** K-Fold Split Results ***
{'0': {'precision': 0.9132149901380671, 'recall': 0.9213930348258706, 'f1-score': 0.9172857850420999, 'support': 1005}, '1': {'precision': 0.9206827309236948, 'recall': 0.9124378109452737, 'f1-score': 0.9165417291354323, 'support': 1005}, 'accuracy': 0.9169154228855722, 'macro avg': {'precision': 0.9169488605308809, 'recall': 0.9169154228855722, 'f1-score': 0.9169137570887661, 'support': 2010}, 'weighted avg': 

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8980769230769231, 'recall': 0.9302788844621513, 'f1-score': 0.913894324853229, 'support': 1004}, '1': {'precision': 0.9277605779153767, 'recall': 0.8945273631840795, 'f1-score': 0.9108409321175277, 'support': 1005}, 'accuracy': 0.9123942259830762, 'macro avg': {'precision': 0.9129187504961499, 'recall': 0.9124031238231154, 'f1-score': 0.9123676284853783, 'support': 2009}, 'weighted avg': {'precision': 0.9129261381653482, 'recall': 0.9123942259830762, 'f1-score': 0.9123668685568727, 'support': 2009}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.9083011583011583, 'recall': 0.9372509960159362, 'f1-score': 0.9225490196078432, 'support': 1004}, '1': {'precision': 0.935251798561151, 'recall': 0.9054726368159204, 'f1-score': 0.9201213346814964, 'support': 1005}, 'accuracy': 0.9213539074166252, 'macro avg': {'precision': 0.9217764784311546, 'recall': 0.9213618164159283, 'f1-score': 0.9213351771446698, 'support': 2009}, 'weighted avg': {'precision': 0.9217831859075759, 'recall': 0.9213539074166252, 'f1-score': 0.9213345729423487, 'support': 2009}}
*** K-Fold Split Results ***
{'0': {'precision': 0.9171656686626747, 'recall': 0.9153386454183267, 'f1-score': 0.9162512462612165, 'support': 1004}, '1': {'precision': 0.9155908639523337, 'recall': 0.9174129353233831, 'f1-score': 0.9165009940357853, 'support': 1005}, 'accuracy': 0.9163763066202091, 'macro avg': {'precision': 0.9163782663075042, 'recall': 0.9163757903708549, 'f1-score': 0.9163761201485009, 'support': 2009}, 'weighted avg': {

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [30]:
# calculate average metrics for submission
import numpy
final_dict = {}
final_dict['0-precision'] = []
final_dict['0-recall'] = []
final_dict['0-f1'] = []
final_dict['1-precision'] = []
final_dict['1-recall'] = []
final_dict['1-f1'] = []
final_dict['accuracy'] = []

for dictionary in classification_report_list:
    for key, inner_dict in dictionary.items():
        if key == '0':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['0-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['0-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['0-f1'].append(inner_dict['f1-score'])
        if key == '1':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['1-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['1-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['1-f1'].append(inner_dict['f1-score'])
        if key == "accuracy":
            final_dict['accuracy'].append(dictionary['accuracy'])


for key in final_dict.keys():
    final_dict[key] = numpy.average(final_dict[key])

print("******** NN FINAL DICT ********")
print(final_dict)

******** NN FINAL DICT ********
{'0-precision': 0.9095488229235198, '0-recall': 0.9210384994020602, '0-f1': 0.9151885904652037, '1-precision': 0.9201794752698322, '1-recall': 0.9082988113879474, '1-f1': 0.9141281012390716, 'accuracy': 0.9146682383668185}


In [31]:
# classification report on CV with VotingClassifier
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, make_scorer

# define model
model = RandomForestClassifier(random_state = 42)

classification_report_list = []
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in cv.split(X, y):
    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("*** K-Fold Split Results ***")
    classification_report_results = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report_results)
    classification_report_list.append(classification_report_results)

*** K-Fold Split Results ***
{'0': {'precision': 0.9104638619201726, 'recall': 0.8398009950248756, 'f1-score': 0.8737060041407867, 'support': 1005}, '1': {'precision': 0.8513388734995383, 'recall': 0.9174129353233831, 'f1-score': 0.8831417624521072, 'support': 1005}, 'accuracy': 0.8786069651741294, 'macro avg': {'precision': 0.8809013677098554, 'recall': 0.8786069651741293, 'f1-score': 0.8784238832964469, 'support': 2010}, 'weighted avg': {'precision': 0.8809013677098555, 'recall': 0.8786069651741294, 'f1-score': 0.8784238832964469, 'support': 2010}}
*** K-Fold Split Results ***
{'0': {'precision': 0.9114194236926361, 'recall': 0.8497512437810946, 'f1-score': 0.8795056642636457, 'support': 1005}, '1': {'precision': 0.8592730661696178, 'recall': 0.9174129353233831, 'f1-score': 0.8873917228103946, 'support': 1005}, 'accuracy': 0.8835820895522388, 'macro avg': {'precision': 0.8853462449311269, 'recall': 0.8835820895522388, 'f1-score': 0.8834486935370202, 'support': 2010}, 'weighted avg': 

In [32]:
# calculate average metrics for submission
import numpy
final_dict = {}
final_dict['0-precision'] = []
final_dict['0-recall'] = []
final_dict['0-f1'] = []
final_dict['1-precision'] = []
final_dict['1-recall'] = []
final_dict['1-f1'] = []
final_dict['accuracy'] = []

for dictionary in classification_report_list:
    for key, inner_dict in dictionary.items():
        if key == '0':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['0-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['0-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['0-f1'].append(inner_dict['f1-score'])
        if key == '1':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['1-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['1-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['1-f1'].append(inner_dict['f1-score'])
        if key == "accuracy":
            final_dict['accuracy'].append(dictionary['accuracy'])


for key in final_dict.keys():
    final_dict[key] = numpy.average(final_dict[key])

print("******** RF FINAL DICT ********")
print(final_dict)

******** RF FINAL DICT ********
{'0-precision': 0.9035907867310111, '0-recall': 0.8663618824866374, '0-f1': 0.8844828509334206, '1-precision': 0.8718672346884717, '1-recall': 0.9074679391885195, '1-f1': 0.8892238602355227, 'accuracy': 0.8869151092059199}
